### Importere nødvendige biblioteker

In [1]:
import csv
import getopt
import json
import sys
import pandas as pd
from urllib.request import Request, urlopen  # Python 3

In [66]:
api_key = "cEdnUBwYRUanB50ae573FA=="
url = "https://hydapi.nve.no//api/v1/Stations?CountyName=Vestland&Active=0"

request_headers = {
        "Accept": "application/json",
        "X-API-Key": api_key
        }

In [67]:
request = Request(url, headers=request_headers) # Send spørring til hydAPI
print
response = urlopen(request)
content = response.read().decode('utf-8')

df = pd.json_normalize(json.loads(content), 'data')

### Hente observasjoner fra stasjoner i Vestland

In [107]:
station_list = df['stationId'].tolist()
param_list = "1001"

station_chunks = [station_list[i:i + 10] for i in range(0, len(station_list), 10)]

In [175]:
df_list = []

request_headers = {
        "Accept": "application/json",
        "X-API-Key": api_key
        }

for i in station_chunks[0:3]:
    url = ("https://hydapi.nve.no//api/v1/Observations"
        "?StationId={string}&Parameter={param_list}&ResolutionTime=0")
    url_q = url.format(string=','.join(i), param_list = param_list)
    
    print(url_q)
    request_q = Request(url_q, headers=request_headers)

    response_q = urlopen(request_q)
    content_q = response_q.read().decode('utf-8')
    
    data = pd.json_normalize(json.loads(content_q)['data'])
    data = data.explode('observations')
    data = pd.concat([data, data['observations'].apply(pd.Series)], axis=1)
    data = data.drop(columns='observations')
    df_list.append(data)

https://hydapi.nve.no//api/v1/Observations?StationId=12.162.0,15.118.2,36.14.0,36.15.0,36.18.0,36.21.0,36.22.0,36.23.0,36.24.0,36.9.0&Parameter=1001&ResolutionTime=0
https://hydapi.nve.no//api/v1/Observations?StationId=41.1.0,41.12.0,41.2.0,41.34.0,41.36.0,41.37.0,41.38.0,41.4.0,41.8.0,41.9.0&Parameter=1001&ResolutionTime=0
https://hydapi.nve.no//api/v1/Observations?StationId=42.1.0,42.10.0,42.11.0,42.16.0,42.2.0,42.23.0,42.7.0,42.9.0,45.4.0,46.10.0&Parameter=1001&ResolutionTime=0


In [176]:
df_q = pd.concat(df_list, ignore_index=True)

### Hente koordinater fra stasjons og andre data fra df